# 04 Simulation-Algebraic


## 1. General purpose non-linear solvers

###

2. Nonlinear Solvers
These solve $\mathbf{f(x)} = \mathbf{0}$ where $\mathbf{f}$ is a nonlinear vector-valued function. Principles often build on linear solvers, using approximations like Jacobians for local convergence; global methods handle multiple roots or basins.
2.1 Local Methods (Derivative-Based)

Underlying Principle: Use Taylor expansion or quasi-Newton approximations to iterate toward a root, requiring good initial guesses; quadratic convergence for Newton's method.
Examples:

Newton-Raphson: Iterates $\mathbf{x}_{k+1} = \mathbf{x}_k - \mathbf{J}^{-1} \mathbf{f(x_k)}$ where $\mathbf{J}$ is the Jacobian; requires Jacobian computation or finite differences.
Quasi-Newton (e.g., Broyden's): Approximates Jacobian updates without full recomputation; good for expensive Jacobians.
Levenberg-Marquardt: Hybrid trust-region for least-squares problems (damps Newton steps).


SciPy Availability:

Available: scipy.optimize.fsolve (hybrid Newton/minpack; finite-difference Jacobian), scipy.optimize.root (supports 'hybr', 'lm', 'broyden1', 'broyden2', 'krylov'), scipy.optimize.newton (scalar only, secant/Newton).
Not Available: Built-in damped Newton variants or automatic differentiation for Jacobians (use JAX or autograd externally); no native homotopy continuation (e.g., for path-tracking multiple roots).



2.2 Global/Derivative-Free Methods

Underlying Principle: Explore search space without derivatives, using heuristics or sampling; slower but robust to poor initials or multimodality.
Examples:

Bracketing Methods: Bisection (scalar, interval-based), Brent's (hybrid secant/bisection).
Population-Based: Genetic algorithms, particle swarm (evolutionary; for global roots).
Trust-Region Dogleg: For unconstrained nonlinear equations.


SciPy Availability:

Available: scipy.optimize.root (with 'df-sane' for derivative-free spectral), scipy.optimize.brentq/brenth (scalar bracketing), scipy.optimize.minimize (can adapt for root-finding via optimization, e.g., 'powell' or 'nelder-mead').
Not Available: Built-in evolutionary/global methods like differential evolution for pure root-finding (though scipy.optimize.differential_evolution exists for optimization); no interval arithmetic solvers (e.g., for guaranteed enclosures; use pyinterval).


## 1. Linear Solvers
These solve $\mathbf{Ax} = \mathbf{b}$ where $\mathbf{A}$ is a matrix, $\mathbf{x}$ is the unknown vector, and $\mathbf{b}$ is a known vector. Principles include factorization (direct) or successive approximations (iterative), with considerations for matrix properties (e.g., sparse, symmetric, positive definite) to ensure stability and efficiency.
###  1.1 Direct Methods

Underlying Principle: Decompose the matrix into factors (e.g., triangular) for exact solution in finite steps, assuming no round-off errors. Efficient for dense matrices of moderate size; sensitive to conditioning and pivoting for numerical stability.
Examples:

#### 1.1.1 Gaussian Elimination / LU Decomposition:
Factorizes $\mathbf{A} = \mathbf{LU}$ (lower/upper triangular), solves via forward/backward substitution. Partial pivoting (PLU) improves stability.

scipy.linalg.solve (general direct solver using LAPACK (Linear Algebra PACKage); handles LU internally), scipy.linalg.lu (explicit LU with pivoting),


The SciPy command for this decomposition is linalg.lu. Such a decomposition is often useful for solving many simultaneous equations where the left-hand side does not change but the right-hand side does. For example, suppose we are going to solve



### 1.1.2 Finding the inverse

In SciPy, the matrix inverse of the NumPy array, A, is obtained using linalg.inv (A), or using A.I if A is a Matrix. For example, let

The following example demonstrates this computation in SciPy

#### 1.1.3 Cholesky Decomposition:
For symmetric positive definite matrices, $\mathbf{A} = \mathbf{LL}^T$ or $\mathbf{UU}^T$; faster than LU (about half the operations).

#### 1.1.4 QR Decomposition:
Orthogonal factorization $\mathbf{A} = \mathbf{QR}$; useful for least-squares or underdetermined systems.

#### 1.1.5 Singular Value Decomposition (SVD):
[VERY IMPORTANT ALGORITHM!]

$\mathbf{A} = \mathbf{U\Sigma V}^T$; handles ill-conditioned or rectangular matrices, provides pseudo-inverse.

#### 1.1.6  band-matrix solvers

Available: scipy.linalg.solve (general direct solver using LAPACK; handles LU internally), scipy.linalg.lu (explicit LU with pivoting), scipy.linalg.cholesky, scipy.linalg.qr, scipy.linalg.svd.
Not Available: Built-in band-matrix solvers (though scipy.linalg.solve_banded exists for banded systems); no native parallel direct solvers (use external libraries like PETSc).


### 1.1.6 Condition exception handling

ill-conditioning (check with numpy.linalg.cond)

### 1.2 Iterative Methods

Underlying Principle: Start with an initial guess and refine iteratively until convergence (e.g., residual below tolerance). Ideal for large sparse systems; convergence depends on preconditioning and matrix spectrum.
Examples:

### 1.2.1 Stationary Methods: Jacobi (diagonal dominance),
### 1.2.2 Gauss-Seidel (updates in-place),
### 1.2.3  Successive Over-Relaxation (SOR; accelerates with relaxation parameter).
### 1.2.4 Krylov Subspace Methods: Conjugate Gradient (CG; for symmetric positive definite),
### 1.2.5 Generalized Minimal Residual (GMRES; for nonsymmetric),
### 1.2.6 Biconjugate Gradient Stabilized (BiCGSTAB).
### 1.2.7 Multigrid Methods: Hierarchical smoothing and coarsening for elliptic PDEs (e.g., geometric/algebraic multigrid).
### 1.2.8 Jacobi/Gauss-Seidel/SOR
(implement manually or use external like PyAMG for multigrid); no native distributed-memory iterative solvers (e.g., for HPC; use PETSc or hypre).

Available (for sparse matrices): scipy.sparse.linalg.cg, scipy.sparse.linalg.gmres, scipy.sparse.linalg.bicgstab, scipy.sparse.linalg.minres (Minimum Residual for symmetric indefinite).
Preconditioners via scipy.sparse.linalg.spilu (incomplete LU).
Not Available: Built-in Jacobi/Gauss-Seidel/SOR (implement manually or use external like PyAMG for multigrid); no native distributed-memory iterative solvers (e.g., for HPC; use PETSc or hypre).

In [2]:
import numpy as np
from scipy import linalg

# Example matrix (slightly modified to avoid singularity)
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])

# Compute SVD: Returns U, singular values S (as vector), and Vt (V transpose)
U, S, Vt = linalg.svd(A)

# Display results
print("U matrix:")
print(U)
print("\nSingular values:")
print(S)
print("\nVt matrix:")
print(Vt)

# Reconstruct A to verify (U * diag(S) * Vt)
reconstructed_A = np.dot(U, np.dot(np.diag(S), Vt))
print("\nReconstructed A:")
print(np.round(reconstructed_A, decimals=10))  # Round for floating-point precision

U matrix:
[[-0.20933734  0.96438514  0.16167618]
 [-0.50384851  0.03532145 -0.86306956]
 [-0.8380421  -0.26213299  0.47850992]]

Singular values:
[17.41250517  0.87516135  0.19686652]

Vt matrix:
[[-0.46466755 -0.55375455 -0.69097031]
 [-0.83328635  0.00949949  0.55275999]
 [ 0.2995295  -0.83262576  0.46585022]]

Reconstructed A:
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8. 10.]]


In practice, for solving systems, you might use the pseudo-inverse from SVD for least-squares: pinv_A = np.dot(Vt.T, np.dot(np.diag(1/S), U.T))

In [3]:
pinv_A = np.dot(Vt.T, np.dot(np.diag(1/S), U.T))

3. Special-Purpose Solvers
These handle structured problems beyond standard linear/nonlinear, often integrating with the above.
3.1 Eigenvalue Solvers

Underlying Principle: Find $\lambda, \mathbf{v}$ such that $\mathbf{A v} = \lambda \mathbf{v}$; uses iterations like power method or QR algorithm.
Examples: Standard eigenvalue, generalized ($\mathbf{A v} = \lambda \mathbf{B v}$).
SciPy Availability: Available (scipy.linalg.eig, scipy.linalg.eigh for Hermitian, scipy.sparse.linalg.eigs for sparse).

3.2 Least-Squares Solvers

Underlying Principle: Minimize $\|\mathbf{Ax} - \mathbf{b}\|^2$; uses SVD or normal equations.
Examples: Linear (LSQR), nonlinear (curve_fit).
SciPy Availability: Available (scipy.linalg.lstsq, scipy.optimize.least_squares, scipy.optimize.curve_fit).

3.3 Sparse and Large-Scale Extensions

Underlying Principle: Exploit sparsity to reduce storage/computation; iterative with preconditioners.
SciPy Availability: Strong support via scipy.sparse and scipy.sparse.linalg; missing: Advanced algebraic multigrid (use PyAMG).

Recommendations for Course Integration

In Notebooks: Demonstrate with Python examples, e.g., using scipy.linalg.solve for linear systems in Unit 4 (Simulation I – algebraic systems). Include caveats like ill-conditioning (check with numpy.linalg.cond).
What SciPy Excels At: Dense linear algebra and basic nonlinear root-finding; efficient for engineering-scale problems (<10^4 variables).
Limitations and Alternatives: For very large/sparse systems, suggest PETSc or SuiteSparse; for symbolic Jacobians, integrate SymPy (available in the environment). Avoid over-reliance on SciPy for production HPC.